importing libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

data preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/ann

/content/drive/My Drive/Colab Notebooks/ann


In [ ]:
def callingfromsheetthroughapi(SAMPLE_SPREADSHEET_ID, SERVICE_ACCOUNT_FILE):
  from googleapiclient.discovery import build
  from google.oauth2 import service_account
  import pandas as pd
  SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

  creds = None
  creds = service_account.Credentials.from_service_account_file(
          SERVICE_ACCOUNT_FILE, scopes=SCOPES)

  # The ID and range of a sample spreadsheet.
  service = build('sheets', 'v4', credentials=creds)

  # Call the Sheets API
  sheet = service.spreadsheets()
  result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,range='maindataset_2022').execute()
  values = result.get('values', [])
  dataset = pd.DataFrame(values)
  
  # modify the function to read from a local csv file 
  
  return dataset

df=callingfromsheetthroughapi('1xTxPTnJM6kPVqOeIg6yNL7ktvbpLTgDMXhwZJByq0YY','vrse-preprocessing-3a95b11cc6fd.json')


In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,driver,driver_name,team,laps_completed,pitstop_no.,tyre_age,laps_left,position,race_track,total_laps,year,tyre,tyre_changed
1,1,Max Verstappen,RED BULL RACING RBPT,1,1,1,51,1,silverstone,52,2022,soft,medium
2,1,Max Verstappen,RED BULL RACING RBPT,12,2,12,40,1,silverstone,52,2022,medium,medium
3,1,Max Verstappen,RED BULL RACING RBPT,23,3,11,29,6,silverstone,52,2022,medium,hard
4,1,Max Verstappen,RED BULL RACING RBPT,39,4,16,13,8,silverstone,52,2022,hard,soft


In [ ]:
dataset=df.iloc[1:,3:]
dataset.columns =['laps_completed', 'pitstop_no.', 'tyre_age', 'laps_left', 'position', 'race_track', 'total_laps', 'year', 'tyre', 'tyre_changed']
dataset

,laps_completed,pitstop_no.,tyre_age,laps_left,position,race_track,total_laps,year,tyre,tyre_changed
1,1,1,1,51,1,silverstone,52,2022,soft,medium
2,12,2,12,40,1,silverstone,52,2022,medium,medium
3,23,3,11,29,6,silverstone,52,2022,medium,hard
4,39,4,16,13,8,silverstone,52,2022,hard,soft
5,5,1,5,47,4,silverstone,52,2022,medium,medium
...,...,...,...,...,...,...,...,...,...,...
1829,14,1,14,42,1,united_states,56,2019,medium,hard
1830,35,2,21,21,1,united_states,56,2019,hard,medium
1831,11,1,11,45,18,united_states,56,2019,medium,hard
1832,17,1,17,39,16,united_states,56,2019,medium,hard


one-hot encoding racetrack

In [ ]:

dict={
    'belgian':0.5,
    'silverstone':0.3,
    'monza':0.5,
    'australian':0.1,
    'dutch':0.3,
    'austrian':0.2,
    'bahrain':0.3,
    'austin':0.3,
    'canada':0.2,
    'miami':0.3,
    'hungary':0.3,
    'spanish':0.4,
    'abu_dhabi':0.3,
    'french':0.4,
    'united_states':0.3,
    'styrian':0.2,
    'portuguese':0.4,
    'russian':0.2,
    'eifel':0.2,
    'emilia_romagna':0.5,
    'italian':0.5,
    'bahrain':0.3,
    'tuscany':0.5,
    'azerbaijan':0.1,
    'saudi_arabian':0.2,
    'chinese':0.4,
    'japanese':0.5,
    'monaco':0.1,
    'mexican':0.1,
    'brazilian':0.3,
    'singapore':0.2
}

for racetrack,val in dict.items():
  dataset.replace(racetrack,val,inplace=True)

  

convert_dict = {
                'laps_completed':int, 
                'pitstop_no.':int, 
                'tyre_age':int, 
                'laps_left':int, 
                'position':int, 
                'race_track':float, 
                'total_laps':int, 
                'year':int, 
                # 'tyre':int, 
                # 'tyre_changed':int
                }
  
dataset = dataset.astype(convert_dict)

dataset=dataset[dataset['tyre_age'] > 9]

#removing all pitstops made after 80% of a race as it is very unlikely that a pitstop is made after that 
def completion(dataset,completion):
    dataset = dataset[(dataset['laps_completed']) < (completion/100*(dataset['total_laps']))]
    return dataset

dataset=completion(dataset,80)
dataset

,laps_completed,pitstop_no.,tyre_age,laps_left,position,race_track,total_laps,year,tyre,tyre_changed
2,12,2,12,40,1,0.3,52,2022,medium,medium
3,23,3,11,29,6,0.3,52,2022,medium,hard
4,39,4,16,13,8,0.3,52,2022,hard,soft
6,39,2,34,13,4,0.3,52,2022,medium,soft
7,20,1,20,32,1,0.3,52,2022,medium,hard
...,...,...,...,...,...,...,...,...,...,...
1829,14,1,14,42,1,0.3,56,2019,medium,hard
1830,35,2,21,21,1,0.3,56,2019,hard,medium
1831,11,1,11,45,18,0.3,56,2019,medium,hard
1832,17,1,17,39,16,0.3,56,2019,medium,hard


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [-2])], remainder='passthrough')
dataset=np.array(ct.fit_transform(dataset))

In [ ]:
ct=ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [-1])], remainder='passthrough')
dataset=np.array(ct.fit_transform(dataset))

In [ ]:
dataset

array([[0.0, 1.0, 0.0, ..., 0.3, 52, 2022],
       [1.0, 0.0, 0.0, ..., 0.3, 52, 2022],
       [0.0, 0.0, 1.0, ..., 0.3, 52, 2022],
       ...,
       [1.0, 0.0, 0.0, ..., 0.3, 56, 2019],
       [1.0, 0.0, 0.0, ..., 0.3, 56, 2019],
       [0.0, 0.0, 1.0, ..., 0.3, 56, 2019]], dtype=object)

In [ ]:
dataset = np.asarray(dataset).astype('float32')

dataset[:5,:]


array([[0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        1.200e+01, 2.000e+00, 1.200e+01, 4.000e+01, 1.000e+00, 3.000e-01,
        5.200e+01, 2.022e+03],
       [1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        2.300e+01, 3.000e+00, 1.100e+01, 2.900e+01, 6.000e+00, 3.000e-01,
        5.200e+01, 2.022e+03],
       [0.000e+00, 0.000e+00, 1.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
        3.900e+01, 4.000e+00, 1.600e+01, 1.300e+01, 8.000e+00, 3.000e-01,
        5.200e+01, 2.022e+03],
       [0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        3.900e+01, 2.000e+00, 3.400e+01, 1.300e+01, 4.000e+00, 3.000e-01,
        5.200e+01, 2.022e+03],
       [1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        2.000e+01, 1.000e+00, 2.000e+01, 3.200e+01, 1.000e+00, 3.000e-01,
        5.200e+01, 2.022e+03]], dtype=float32)

data encoding- next tire values(output/independent variable)

In [ ]:
X = dataset[:, 3:]
y = dataset[:, :3]

In [ ]:
X

array([[0.000e+00, 1.000e+00, 0.000e+00, ..., 3.000e-01, 5.200e+01,
        2.022e+03],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 3.000e-01, 5.200e+01,
        2.022e+03],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 3.000e-01, 5.200e+01,
        2.022e+03],
       ...,
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 3.000e-01, 5.600e+01,
        2.019e+03],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 3.000e-01, 5.600e+01,
        2.019e+03],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 3.000e-01, 5.600e+01,
        2.019e+03]], dtype=float32)

In [ ]:
y

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

splitting the dataset into training and testing

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 32)

In [ ]:
X_train[0]

array([1.000e+00, 0.000e+00, 0.000e+00, 2.000e+01, 2.000e+00, 1.900e+01,
       3.700e+01, 1.600e+01, 3.000e-01, 5.700e+01, 2.021e+03],
      dtype=float32)

feature scaling for normalization

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

artificial neural network

initialization of ann

In [ ]:
#vrse created as a neural network which is an instance of the sequential class
from keras.layers import Dropout
vrse = tf.keras.models.Sequential()

adding the input and hidden layer

In [ ]:
vrse.add(tf.keras.layers.Dense(units=512, activation='relu'))

adding a dropout layer 20%

In [ ]:
vrse.add(Dropout(0.2))

adding second hidden layer

In [ ]:
vrse.add(tf.keras.layers.Dense(units=256, activation='relu'))

In [ ]:
vrse.add(Dropout(0.2))

In [ ]:
vrse.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [ ]:
vrse.add(Dropout(0.2))

In [ ]:
vrse.add(tf.keras.layers.Dense(units=64, activation='relu'))

In [ ]:
vrse.add(Dropout(0.2))

In [ ]:
vrse.add(tf.keras.layers.Dense(units=32, activation='relu'))

In [ ]:
vrse.add(Dropout(0.2))

In [ ]:
vrse.add(tf.keras.layers.Dense(units=16, activation='relu'))

Adding the output layer

In [ ]:
vrse.add(tf.keras.layers.Dense(units=3, activation='sigmoid'))

training the neural network

compiling the neural network

In [ ]:
# adam optimizer chosen to perform stochastic gradient descent
# categorical_crossentropy chosen as loss function to compute difference between real and predicted for binary outcomes
vrse.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

training with training set

In [ ]:
vrse.fit(X_train, y_train, batch_size=32, epochs=90)

Epoch 1/90
33/33 [==============================] - 2s 8ms/step - loss: 0.9822 - accuracy: 0.5448
Epoch 2/90
33/33 [==============================] - 0s 9ms/step - loss: 0.8259 - accuracy: 0.6657
Epoch 3/90
33/33 [==============================] - 0s 8ms/step - loss: 0.7647 - accuracy: 0.6881
Epoch 4/90
33/33 [==============================] - 0s 8ms/step - loss: 0.7574 - accuracy: 0.6949
Epoch 5/90
33/33 [==============================] - 0s 8ms/step - loss: 0.7108 - accuracy: 0.7164
Epoch 6/90
33/33 [==============================] - 0s 8ms/step - loss: 0.6791 - accuracy: 0.7456
Epoch 7/90
33/33 [==============================] - 0s 9ms/step - loss: 0.6733 - accuracy: 0.7251
Epoch 8/90
33/33 [==============================] - 0s 8ms/step - loss: 0.6630 - accuracy: 0.7339
Epoch 9/90
33/33 [==============================] - 0s 8ms/step - loss: 0.6355 - accuracy: 0.7281
Epoch 10/90
33/33 [==============================] - 0s 9ms/step - loss: 0.6025 - accuracy: 0.7544
Epoch 11/90
33/33 [

testing 

In [ ]:
y_pred=(vrse.predict(X_test))
for i in y_pred:
  i[i == max(i)] = 1
  i[i<1] = 0

14/14 [==============================] - 0s 7ms/step


In [ ]:
y_pred

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
y_test

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [ ]:
print(np.concatenate((y_pred, y_test), axis=1))

[[1. 0. 0. 1. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0. 1.]
 ...
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 1. 0.]]



accuracy score

In [ ]:
y_pred=pd.DataFrame(y_pred)
y_test=pd.DataFrame(y_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7505668934240363

In [ ]:
import joblib
filename = 'ann4.h5'
vrse.save('ann4.h5')
joblib.dump(sc,'scaler.sav')

['scaler.sav']

In [ ]:
from tensorflow import keras
loaded_model = keras.models.load_model('ann4.h5')
result = (loaded_model.predict(X_test))
for i in result:
  i[i == max(i)] = 1
  i[i<1] = 0
print(accuracy_score(y_test, result))

14/14 [==============================] - 0s 3ms/step
0.7505668934240363


In [ ]:
# 12	2	12	40	1	0.3	52	2022	medium	
scale_model = joblib.load('scaler.sav')

a=12
b=2
c=12
d=40
e=1
f=0.3
g=52
k=2022
h=0
i=0
j=1



arr=np.array([[h,i,j,a,b,c,d,e,f,g,k]])
arr = scale_model.transform(arr)
arr = vrse.predict(arr)
for i in arr:
  i[i == max(i)] = 1
  i[i<1] = 0
arr

1/1 [==============================] - 0s 25ms/step


array([[0., 1., 0.]], dtype=float32)